# Facility Types Issue

Cross-referencing the model data with food inspection records from the Chicago data portal suggests that the model train/test data includes many different facility types. Among others, the data appears to include hospitals and schools, which [the paper](https://github.com/Chicago/food-inspections-evaluation/blob/master/REPORTS/forecasting-restaurants-with-critical-violations-in-Chicago.pdf) specifically states should be excluded.

This notebook cross-references `DATA/30_glmnet_data.Rds`, the data file produced by running `CODE/30_glmnet_model.R`, with the public dataset of [food inspection records](https://data.cityofchicago.org/Health-Human-Services/Food-Inspections/4ijn-s7e5) on the Chicago data portal.

- Is there a discrepancy between how food establishments are classified in the data portal and how they are classified in their business license?

There are at least two locations in the code that appear to filter out other types of facilities:

- [`CODE/23_generate_model_dat.R` line 51](https://github.com/Chicago/food-inspections-evaluation/blob/master/CODE/23_generate_model_dat.R#L51)
- [`CODE/30_glmnet_model.R` line 23](https://github.com/Chicago/food-inspections-evaluation/blob/master/CODE/30_glmnet_model.R#L23)

In [1]:
# Notebook dependencies
!pip install pandas
!pip install numpy
!pip install requests
!pip install pyreadr
import pandas as pd
import numpy as np
import requests
import pyreadr

You are using pip version 9.0.3, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


Download my copy of `30_glmnet_data.Rds`. Substitute with your own version of the file, or follow these instructions from the [original GitHub repository](https://github.com/Chicago/food-inspections-evaluation#file-layout) to reproduce.

```
# In the directory: food_inspections_evaluation
$ RScript CODE/00_Startup.R
$ RScript CODE/30_glmnet_model.R
```

In [2]:
GLMNET_DATA_URL = "https://raw.githubusercontent.com/vingkan/foodinspectionforecasting/master/DATA/30_glmnet_data.Rds"
GLMNET_DOWNLOADED_PATH = "downloaded_30_glmnet_data.Rds"
res = requests.get(GLMNET_DATA_URL)
with open(GLMNET_DOWNLOADED_PATH, "wb") as file:
    file.write(res.content)

In [3]:
raw_dat = pyreadr.read_r(GLMNET_DOWNLOADED_PATH)[None]
print("GLMNet table contains {} records.".format(len(raw_dat)))
raw_dat["inspection_id_str"] = raw_dat["Inspection_ID"].astype(int).astype(str)
print("GLMNet table contains {} unique inspection IDs.".format(len(raw_dat["inspection_id_str"].unique())))
print("GLMNet table contains {} unique license IDs.".format(len(raw_dat["License"].unique())))

GLMNet table contains 18712 records.
GLMNet table contains 18712 unique inspection IDs.
GLMNet table contains 9710 unique license IDs.


In [4]:
raw_dat.head()

,Inspection_Date,License,Inspection_ID,Business_ID,criticalCount,seriousCount,minorCount,Facility_Type,pass_flag,fail_flag,...,Inspector_Assigned,precipIntensity,temperatureMax,windSpeed,humidity,criticalFound,score,Test,Train,inspection_id_str
0,15736,30790,269961.0,30790-20110416,0.0,0.0,2.0,Grocery_Store,1.0,0.0,...,green,0.014587,53.496667,13.340000,0.900000,0.0,0.098925,False,True,269961
1,15265,1475890,507211.0,1475890-20110416,0.0,0.0,3.0,Restaurant,1.0,0.0,...,blue,0.001907,59.046667,13.016667,0.550000,0.0,0.225081,False,True,507211
2,15265,1740130,507212.0,1740130-20110216,0.0,2.0,6.0,Restaurant,0.0,1.0,...,blue,0.001907,59.046667,13.016667,0.550000,0.0,0.225782,False,True,507212
3,15266,1447363,507216.0,1447363-20110216,0.0,0.0,6.0,Restaurant,1.0,0.0,...,blue,0.002737,56.153333,10.863333,0.616667,0.0,0.229768,False,True,507216
4,15267,1679459,507219.0,1679459-20100216,0.0,2.0,6.0,Restaurant,0.0,1.0,...,blue,0.009987,52.730000,16.266667,0.690000,0.0,0.220462,False,True,507219


According to the `Facility_Type` column in this data table, there are `1003` facilities with types other than restaurant or grocery store.

In [5]:
facility_type_counts = raw_dat["Facility_Type"].value_counts()
print("Counts of facility type over {} inspections:".format(facility_type_counts.sum()))
print()
print(facility_type_counts)

Counts of facility type over 18712 inspections:

Restaurant       15115
Grocery_Store     2594
Other             1003
Name: Facility_Type, dtype: int64


Retrieve all canvass inspections from the data portal during the time range specified in the paper. This should return `41366` rows.

In [6]:
INSPECTIONS_DATA_URL = "https://data.cityofchicago.org/resource/cwig-ma7x.json"
query = """
    SELECT
        address AS address,
        aka_name AS aka_name,
        city AS city,
        dba_name AS dba_name,
        facility_type AS facility_type,
        inspection_date AS inspection_date,
        inspection_id AS inspection_id,
        inspection_type AS inspection_type,
        latitude AS latitude,
        license_ AS license_id,
        longitude AS longitude,
        results AS results,
        risk AS risk,
        state AS state,
        violations AS violations,
        zip AS zip
    WHERE
        inspection_type = "Canvass"
        AND inspection_date >= "2011-01-01"
        AND inspection_date <= "2014-11-01"
    LIMIT 100000
"""


r = requests.get(INSPECTIONS_DATA_URL, params={"$query": query})
rows = r.json()
expected = 41366 # Expect 41366 rows
print("Fetched {} rows from data portal inspections. Expected {}. Match? {}".format(len(rows), expected, len(rows) == expected))
inspecs = pd.DataFrame(rows)

Fetched 41366 rows from data portal inspections. Expected 41366. Match? True


Join the data table with the data portal records using Pandas join.

In [7]:
merged = raw_dat.set_index("inspection_id_str").join(inspecs.set_index("inspection_id"))
print("Merged table contains {} records.".format(len(merged)))

Merged table contains 18712 records.


According to the `facility_type` column in the data portal records, the train/test data includes facility types such as schools and hospitals, among others.

There also appear to be `11` inspection IDs from the model data that were not returned in the data portal query.

In [8]:
facility_types_data_portal = merged["facility_type"]
print("Merged table contains {} unique facility types:".format(len(facility_types_data_portal.unique())))
print()
print(facility_types_data_portal.value_counts())

Merged table contains 141 unique facility types:

Restaurant                                    15080
Grocery Store                                  2570
Bakery                                          379
School                                          148
Catering                                        142
Hospital                                         47
BANQUET HALL                                     16
Long Term Care                                   15
Liquor                                           14
GAS STATION                                      11
STADIUM                                          10
Shelter                                          10
CAFETERIA                                         8
Wholesale                                         7
GROCERY/RESTAURANT                                7
LIVE POULTRY                                      6
Special Event                                     5
Grocery & Restaurant                              4
BANQUET       

In [9]:
def is_other_facility(val):
    try:
        text = val.lower()
        is_restaurant = "restaurant" in text
        is_grocery = "grocery" in text
        return not (is_restaurant or is_grocery)
    except:
        print("Value is not a string: {}".format(val))
        return False
    
other_facilities = list(filter(lambda val: is_other_facility(val), facility_types_data_portal))
print()
msg = "Approximately {} records appear to be facility types other than restaurant/grocery store."
print(msg.format(len(other_facilities)))


Value is not a string: nan
Value is not a string: nan
Value is not a string: nan
Value is not a string: nan
Value is not a string: nan
Value is not a string: nan
Value is not a string: nan
Value is not a string: nan
Value is not a string: nan
Value is not a string: nan
Value is not a string: nan

Approximately 994 records appear to be facility types other than restaurant/grocery store.


As a sanity check, to be sure there was nothing wrong with the join, find the facility_type for each inspection ID.

In [10]:
inspection_map = {}
for row in rows:
    if "facility_type" in row:
        inspection_id = row["inspection_id"]
        facility_type = row["facility_type"]
        if inspection_id in inspection_map:
            print("DUPLICATE INSPECTION_ID: {}".format(inspection_id))
        inspection_map[inspection_id] = facility_type

facility_types = []
for inspection_id in raw_dat["inspection_id_str"].values:
    if inspection_id not in inspection_map:
        print("NOT FOUND IN QUERY RESULTS: {}".format(inspection_id))
        continue
    facility_types.append(inspection_map[inspection_id])
print()
print("Query results contain {} unique facility types.".format(len(np.unique(facility_types))))
print()
print(pd.Series(facility_types).value_counts())

NOT FOUND IN QUERY RESULTS: 569761
NOT FOUND IN QUERY RESULTS: 606447
NOT FOUND IN QUERY RESULTS: 608301
NOT FOUND IN QUERY RESULTS: 610285
NOT FOUND IN QUERY RESULTS: 635049
NOT FOUND IN QUERY RESULTS: 1092477
NOT FOUND IN QUERY RESULTS: 1138542
NOT FOUND IN QUERY RESULTS: 1230083
NOT FOUND IN QUERY RESULTS: 1236048
NOT FOUND IN QUERY RESULTS: 1322477
NOT FOUND IN QUERY RESULTS: 1439578

Query results contain 140 unique facility types.

Restaurant                                    15080
Grocery Store                                  2570
Bakery                                          379
School                                          148
Catering                                        142
Hospital                                         47
BANQUET HALL                                     16
Long Term Care                                   15
Liquor                                           14
GAS STATION                                      11
STADIUM                               

In [11]:
other_facilities_check = list(filter(lambda val: is_other_facility(val), facility_types))
msg = "Approximately {} records appear to be facility types other than restaurant/grocery store."
print(msg.format(len(other_facilities_check)))

Approximately 994 records appear to be facility types other than restaurant/grocery store.
